In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process 
import datetime as dt

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [26]:
dhc = pd.read_csv('DHC_data/DHC_data.csv')
# wslive = pd.read_csv('Data/wslive_with_results_2019-06-07_to_2020-02-21.csv')

In [122]:
['ws_year','ws_month']
phone = good_phone[(good_phone.ws_year==2020)&(good_phone.ws_month==1)]
address = good_address[(good_address.ws_year==2020)&(good_address.ws_month==1)]

In [104]:
good_phone = wslive[(wslive['OFFICE_PHONE_VERIFIED_UPDATED'].isin([1,2])) & (wslive['WSLIVE_SOURCE']=='UPDATED')]
good_address = wslive[(wslive['OFFICE_ADDRESS_VERIFIED_UPDATED'].isin([1,2])) & (wslive['WSLIVE_SOURCE']=='UPDATED')]

In [20]:
phone = good_phone[['PHYSICIAN_ME_NUMBER', 'PHYSICIAN_FIRST_NAME', 'PHYSICIAN_MIDDLE_NAME',
       'PHYSICIAN_LAST_NAME', 'SUFFIX', 'DEGREE', 'OFFICE_ADDRESS_LINE_1',
       'OFFICE_ADDRESS_LINE_2', 'OFFICE_ADDRESS_CITY', 'OFFICE_ADDRESS_STATE',
       'OFFICE_ADDRESS_ZIP', 'OFFICE_TELEPHONE', 'WSLIVE_FILE_DT']]
address = good_address[['PHYSICIAN_ME_NUMBER', 'PHYSICIAN_FIRST_NAME', 'PHYSICIAN_MIDDLE_NAME',
       'PHYSICIAN_LAST_NAME', 'SUFFIX', 'DEGREE', 'OFFICE_ADDRESS_LINE_1',
       'OFFICE_ADDRESS_LINE_2', 'OFFICE_ADDRESS_CITY', 'OFFICE_ADDRESS_STATE',
       'OFFICE_ADDRESS_ZIP', 'OFFICE_TELEPHONE', 'WSLIVE_FILE_DT']]

In [27]:
dhc = dhc.rename(columns ={'Additional Office Locations':'Add_Offices', 'Phone Number':'Phone_Number', 'Zip Code':'Zipcode_DHC', 'First Name':'First', 'Physician Name':'Name_DHC',
       'Middle Name':'Middle', 'Primary Specialty':'Spec', 'Primary Hospital Affiliation':'Affiliation',
       'Last Name':'Last'})

In [143]:
dhc_phone = pd.merge(dhc, GUTE, left_on='ME',right_on='PHYSICIAN_ME_NUMBER')

In [124]:
dhc_address = pd.merge(dhc, address, left_on='ME',right_on='PHYSICIAN_ME_NUMBER')

In [148]:
samp=samp.fillna('None')

In [126]:
dhc_address=dhc_address.fillna('None')

In [166]:
phones=[]
for row in dhc_phone.itertuples():
    phones.append(row.Phone_Number.replace('.',''))

In [167]:
dhc_phone.Phone_Number=phones

In [91]:
def add_match(df):
    count = 0
    for row in df.itertuples():
        match = False
        if row.OFFICE_ADDRESS_LINE_2 in row.Address.upper():
            match = True
        if row.Address.upper() in row.OFFICE_ADDRESS_LINE_2:
            match = True
        if fuzz.ratio(row.OFFICE_ADDRESS_LINE_2,row.Address.upper())>70:
            match = True
        if match == True:
            count +=1
    return count

def phone_match(df):
    count = 0
    for row in df.itertuples():
        if row.Phone_Number == str(int(row.OFFICE_TELEPHONE)):
            count+=1
    return count

def zip_match(df):
    count = 0
    for row in df.itertuples():
        if row.Zipcode_DHC == row.OFFICE_ADDRESS_ZIP:
            count+=1
    return count

def state_match(df):
    count = 0
    for row in df.itertuples():
        if row.State == row.OFFICE_ADDRESS_STATE:
            count+=1
    return count

def city_match(df):
    count = 0
    for row in df.itertuples():
        if row.City.upper() == row.OFFICE_ADDRESS_CITY:
            count+=1
    return count

In [96]:
def return_counts(phone_df, add_df):
    p =phone_match(phone_df)
    a =add_match(add_df)
    z =zip_match(add_df)
    c =city_match(add_df)
    s = state_match(add_df)
    print(f'{p} ({round(p/len(phone_df)*100,2)}%) phones match')
    print(f'{a} ({round(a/len(add_df)*100,2)}%) addresses match')
    print(f'{z} ({round(z/len(add_df)*100,2)}%) zipcodes match')
    print(f'{c} ({round(c/len(add_df)*100,2)}%) cities match')
    print(f'{s} ({round(s/len(add_df)*100,2)}%) states match')

In [187]:
# samp = dhc_phone.sample(500)
return_counts(sample, sample)

311 (62.2%) phones match
341 (68.2%) addresses match
339 (67.8%) zipcodes match
376 (75.2%) cities match
459 (91.8%) states match


In [132]:
len(dhc_address)

4799

In [100]:
good_phone.groupby(['ws_year','ws_month']).count()

PHYSICIAN_ME_NUMBER  PHYSICIAN_FIRST_NAME  \
ws_year ws_month                                              
2019    6                       13178                 13178   
        7                       12021                 12021   
        8                       12549                 12549   
        9                        8659                  8659   
        10                      10347                 10346   
        11                       9761                  9760   
        12                       8935                  8935   
2020    1                        6919                  6918   
        2                        6360                  6360   

                  PHYSICIAN_MIDDLE_NAME  PHYSICIAN_LAST_NAME  SUFFIX  DEGREE  \
ws_year ws_month                                                               
2019    6                         11124                13178     425       0   
        7                         10133                12021     393       0   
        8                         10397                12549     385       0   
        9                          7190                 8658     258       0   
        10                         8510                10347     314       0   
        11                         7933                 9761     262       0   
        12                         7246                 8935     249       0   
2020    1                          5732                 6919     205       0   
        2                          5373                 6360     234       0   

                  OFFICE_ADDRESS_LINE_1  OFFICE_ADDRESS_LINE_2  \
ws_year ws_month                                                 
2019    6                          7328                  13178   
        7                          4229                  12021   
        8                          4453                  12549   
        9                          2925                   8659   
        10                         2944                  10347   
        11                         2851                   9760   
        12                         2600                   8935   
2020    1                          2233                   6919   
        2                          1772                   6360   

                  OFFICE_ADDRESS_CITY  OFFICE_ADDRESS_STATE  ...  \
ws_year ws_month                                             ...   
2019    6                       13178                 13178  ...   
        7                       12021                 12021  ...   
        8                       12549                 12549  ...   
        9                        8659                  8659  ...   
        10                      10347                 10347  ...   
        11                       9759                  9759  ...   
        12                       8932                  8932  ...   
2020    1                        6919                  6919  ...   
        2                        6360                  6360  ...   

                  MATCH_ADDR_LONG  SPECIALTY  SPECIALTY_UPDATED  \
ws_year ws_month                                                  
2019    6                       0      13178              13178   
        7                       0      12021              12021   
        8                       0      12549              12549   
        9                       0       8659               8659   
        10                      0      10347              10347   
        11                      0       9761               9761   
        12                      0       8935               8935   
2020    1                       0       6919               6919   
        2                       0       6360               6360   

                  SOURCE_FILE_DT  new_metric  addr_status  pe_status  \
ws_year ws_month                                                       
2019    6                      0       13178        13178      13178   
        7    

,PHYSICIAN_ME_NUMBER,PHYSICIAN_FIRST_NAME,PHYSICIAN_MIDDLE_NAME,PHYSICIAN_LAST_NAME,SUFFIX,DEGREE,OFFICE_ADDRESS_LINE_1,OFFICE_ADDRESS_LINE_2,OFFICE_ADDRESS_CITY,OFFICE_ADDRESS_STATE,...,SPECIALTY_UPDATED,SOURCE_FILE_DT,ws_month,ws_year,new_metric,addr_status,pe_status,phone_status,fax_status,spec_status
299577,2.640691e+10,YURI,EDUARDO,CORRALES,NaN,NaN,URB COUNTRY CLUB,820 AVE ITURREGUI,SAN JUAN,PR,...,1.0,2019-12-19,2,2020,Updated,Confirmed,Updated,Confirmed,Confirmed,Confirmed
299578,2.512102e+09,CASEY,CATHERINE,SAGER,NaN,NaN,NaN,3409 LUDINGTON ST STE 204,ESCANABA,MI,...,1.0,2019-12-19,2,2020,Updated,Confirmed,Confirmed,Confirmed,Updated,Confirmed
299579,1.102932e+09,CATHERINE,C,MADAFFARI,NaN,NaN,COMM INTERNAL MED,4500 SAN PABLO RD S,JACKSONVILLE,FL,...,1.0,2019-12-19,2,2020,Updated,Confirmed,Updated,Confirmed,Confirmed,Confirmed
299580,2.401021e+09,CATHERINE,PAGE,CROSLAND,NaN,NaN,UNITY HEALTH CARE INC,425 2ND ST NW,WASHINGTON,DC,...,1.0,2019-12-19,2,2020,Updated,Confirmed,Confirmed,Confirmed,Confirmed,Confirmed
299581,4.301000e+08,CAROLYN,M,BOLTIN,NaN,NaN,NaN,525 E 68TH ST,NEW YORK,NY,...,1.0,2019-12-19,2,2020,Updated,Confirmed,Confirmed,Updated,Updated,Confirmed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310150,4.950295e+10,ROOPALI,NaN,AGGARWAL,NaN,NaN,NaN,1 WEXFORD DR,MONMOUTH JUNCTION,NJ,...,1.0,2020-01-27,2,2020,Updated,Updated,Updated,Confirmed,Confirmed,Confirmed
310151,5.104770e+09,ROBERT,CHILES,ALVIS,NaN,NaN,NaN,3468 PARK AVE,BRONX,NY,...,1.0,2020-01-27,2,2020,Updated,Updated,Updated,Confirmed,Updated,Confirmed
310152,7.029603e+08,PATRICK,CHARLES,BARRY,NaN,NaN,NaN,45 COUNTY ROAD 804 STE 210,FRASER,CO,...,1.0,2020-01-27,2,2020,Updated,Updated,Confirmed,Confirmed,Confirmed,Confirmed
310153,3.050101e+10,ROHIT,NaN,KHIRBAT,NaN,NaN,NaN,9901 MEDICAL CENTER DR,ROCKVILLE,MD,...,1.0,2020-01-27,2,2020,Updated,Updated,Updated,Confirmed,Updated,Confirmed


In [134]:
good_phone.WS

,PHYSICIAN_ME_NUMBER,PHYSICIAN_FIRST_NAME,PHYSICIAN_MIDDLE_NAME,PHYSICIAN_LAST_NAME,SUFFIX,DEGREE,OFFICE_ADDRESS_LINE_1,OFFICE_ADDRESS_LINE_2,OFFICE_ADDRESS_CITY,OFFICE_ADDRESS_STATE,...,SPECIALTY_UPDATED,SOURCE_FILE_DT,ws_month,ws_year,new_metric,addr_status,pe_status,phone_status,fax_status,spec_status
0,1.002122e+09,TRAVIS,JOHN,MAIERS,NaN,NaN,NaN,1 CAPITAL WAY,PENNINGTON,NJ,...,1.0,NaN,6,2019,Updated,Confirmed,Updated,Confirmed,Updated,Confirmed
1,1.222090e+09,WILLIAM,HENRY,MCRAE,III,NaN,STE 1400,471 E BROAD ST,COLUMBUS,OH,...,1.0,NaN,6,2019,Updated,Confirmed,Confirmed,Confirmed,Updated,Confirmed
2,1.606101e+09,WHITNEY,ELIZABETH,ZIRKLE,NaN,NaN,STE 1325,675 N SAINT CLAIR ST,CHICAGO,IL,...,1.0,NaN,6,2019,Updated,Confirmed,Updated,Confirmed,Updated,Confirmed
3,1.643631e+09,WALTER,CHAS,WROBEL,NaN,NaN,NaN,623 E MAUDE AVE,ARLINGTON HEIGHTS,IL,...,1.0,NaN,6,2019,Updated,Confirmed,Updated,Confirmed,Updated,Confirmed
4,2.002112e+09,SUSAN,BERSOT,VINCI,NaN,NaN,NaN,200 ABRAHAM FLEXNER WAY,LOUISVILLE,KY,...,1.0,NaN,6,2019,Updated,Confirmed,Updated,Confirmed,Updated,Confirmed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310150,4.950295e+10,ROOPALI,NaN,AGGARWAL,NaN,NaN,NaN,1 WEXFORD DR,MONMOUTH JUNCTION,NJ,...,1.0,2020-01-27,2,2020,Updated,Updated,Updated,Confirmed,Confirmed,Confirmed
310151,5.104770e+09,ROBERT,CHILES,ALVIS,NaN,NaN,NaN,3468 PARK AVE,BRONX,NY,...,1.0,2020-01-27,2,2020,Updated,Updated,Updated,Confirmed,Updated,Confirmed
310152,7.029603e+08,PATRICK,CHARLES,BARRY,NaN,NaN,NaN,45 COUNTY ROAD 804 STE 210,FRASER,CO,...,1.0,2020-01-27,2,2020,Updated,Updated,Confirmed,Confirmed,Confirmed,Confirmed
310153,3.050101e+10,ROHIT,NaN,KHIRBAT,NaN,NaN,NaN,9901 MEDICAL CENTER DR,ROCKVILLE,MD,...,1.0,2020-01-27,2,2020,Updated,Updated,Updated,Confirmed,Updated,Confirmed


In [142]:
GUTE = good_phone[good_phone.WSLIVE_FILE_DT=='2020-02-14'].drop_duplicates('OFFICE_TELEPHONE')

In [145]:
dhc_phone.sample(500)

,NPI,ME,Add_Offices,Phone_Number,Zipcode_DHC,State,City,Address,First,Name_DHC,...,SPECIALTY_UPDATED,SOURCE_FILE_DT,ws_month,ws_year,new_metric,addr_status,pe_status,phone_status,fax_status,spec_status
2451,1689056632,1.275150e+09,NaN,517.205.4730,49201,MI,Jackson,205 N East Ave,Farhad,Farhad Amani,...,1.0,2020-01-27,2,2020,Updated,Confirmed,Confirmed,Confirmed,Updated,Confirmed
403,1275587578,3.840812e+09,NaN,614.221.3777,43222,OH,Columbus,51 S Souder Ave,James,James Sides,...,1.0,2020-01-27,2,2020,Updated,Confirmed,Confirmed,Confirmed,Confirmed,Confirmed
1297,1932155884,7.040197e+10,NaN,972.747.6042,75013,TX,Allen,1105 Central Expy N,Jari,Jari Khan,...,1.0,2020-01-27,2,2020,Updated,Confirmed,Confirmed,Confirmed,Updated,Confirmed
912,1891793287,1.320286e+10,NaN,239.985.1050,33912,FL,Fort Myers,6840 International Center Blvd,Daniel,Daniel Bendetowicz,...,1.0,2020-01-27,2,2020,Updated,Confirmed,Confirmed,Confirmed,Confirmed,Confirmed
978,1659466639,5.611479e+10,"Merrillville, IN - Michigan City, IN",NaN,46383,IN,Valparaiso,85 E US-6 Frontage Rd,Joseph,Joseph Venditti,...,1.0,2020-01-27,2,2020,Updated,Confirmed,Confirmed,Confirmed,Updated,Confirmed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1083,1306892211,6.490381e+10,NaN,239.343.9700,33905,FL,Fort Myers,8960 Colonial Center Dr,John,John Macaluso,...,1.0,2020-01-27,2,2020,Updated,Confirmed,Confirmed,Confirmed,Confirmed,Confirmed
81,1083683833,2.275890e+09,NaN,603.433.8434,"=""03801""",NH,Portsmouth,155 Borthwick Ave,Daniel,Daniel Davies,...,1.0,2020-01-27,2,2020,Updated,Confirmed,Confirmed,Confirmed,Confirmed,Confirmed
193,1710986450,2.507990e+09,NaN,313.875.5377,48202,MI,Detroit,2888 W Grand Blvd,Felicia,Felicia Randolph,...,1.0,2020-01-27,2,2020,Updated,Updated,Confirmed,Updated,Updated,Confirmed
918,1568467447,4.705771e+09,NaN,404.501.5256,30033,GA,Decatur,2701 N Decatur Rd,Edwin,Edwin Nunnery,...,1.0,2020-01-27,2,2020,Updated,Updated,Confirmed,Confirmed,Confirmed,Confirmed


In [160]:
dhc_phone = dhc_phone.fillna('None')

In [165]:
dhc_phone = dhc_phone[(dhc_phone['Phone_Number']!='None')&(dhc_phone['Address']!='None')]

In [170]:
samp = dhc_phone.sample(500)

In [179]:
DHC_PHONE = dhc_phone[['NPI', 'ME', 'Add_Offices', 'Phone_Number', 'Zipcode_DHC', 'State',
       'City', 'Address', 'First', 'Name_DHC', 'Middle', 'Spec', 'Affiliation',
       'Last', 'Address1', 'PHYSICIAN_ME_NUMBER', 'PHYSICIAN_FIRST_NAME',
       'PHYSICIAN_MIDDLE_NAME', 'PHYSICIAN_LAST_NAME', 'SUFFIX', 'DEGREE',
       'OFFICE_ADDRESS_LINE_1', 'OFFICE_ADDRESS_LINE_2', 'OFFICE_ADDRESS_CITY',
       'OFFICE_ADDRESS_STATE', 'OFFICE_ADDRESS_ZIP','OFFICE_TELEPHONE', 'WSLIVE_SOURCE',
       'SPECIALTY']]

In [185]:
sample = DHC_PHONE.sample(500)

In [184]:
DHC_PHONE = DHC_PHONE.drop_duplicates('Phone_Number')

In [186]:
sample.to_csv('Data/dhc_phone_sample.csv', index=False)